In [127]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import random
# Dependencies and Setup
# SQL Alchemy
from sqlalchemy import create_engine

In [128]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [129]:
engine = create_engine("sqlite:///Resources/happiness.sqlite")

In [130]:
df1 = pd.read_csv('Resources/2019.csv')

In [106]:
df1.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [107]:
print(df1.index)

RangeIndex(start=0, stop=156, step=1)


In [108]:
print(df1.columns)

Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')


In [109]:
df1.rename(columns={'Overall rank':'overall_rank', 'Country or region':'country_name', 'Score':'score', 'GDP per capita':'gdp_per_capita',
       'Social support':'social_support', 'Healthy life expectancy':'healthy_life_expectancy',
       'Freedom to make life choices':'freedom_to_make_choices', 'Generosity':'generosity',
       'Perceptions of corruption':'perceptions_of_corruption'}, inplace=True)

In [111]:
df1.shape

(156, 9)

In [112]:
# Return non-NAN rows in each column
print(df1.count())

overall_rank                 156
country_name                 156
score                        156
gdp_per_capita               156
social_support               156
healthy_life_expectancy      156
freedom_to_make_choices      156
generosity                   156
perceptions_of_corruption    156
dtype: int64


In [113]:
# Return total number of rows including NAN in the dataframe
df1.size

1404

In [114]:
df1.describe()

,overall_rank,score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_choices,generosity,perceptions_of_corruption
count,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000
mean,78.500000,5.407096,0.905147,1.208814,0.725244,0.392571,0.184846,0.110603
std,45.177428,1.113120,0.398389,0.299191,0.242124,0.143289,0.095254,0.094538
min,1.000000,2.853000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,39.750000,4.544500,0.602750,1.055750,0.547750,0.308000,0.108750,0.047000
50%,78.500000,5.379500,0.960000,1.271500,0.789000,0.417000,0.177500,0.085500
75%,117.250000,6.184500,1.232500,1.452500,0.881750,0.507250,0.248250,0.141250
max,156.000000,7.769000,1.684000,1.624000,1.141000,0.631000,0.566000,0.453000


In [117]:
df2 = pd.read_csv('Resources/PI_2019_Data.csv')

In [87]:
df2.head()

,area_name,area_code,area_group,pillar_name,element_name,rank_2009,rank_2010,rank_2011,rank_2012,rank_2013,...,score_2010,score_2011,score_2012,score_2013,score_2014,score_2015,score_2016,score_2017,score_2018,score_2019
0,Afghanistan,AFG,Asia-Pacific,Economic Quality,Dynamism,145,146,148,133,131,...,2.2,2.1,2.9,3.1,2.6,3.4,2.4,2.3,2.3,2.5
1,Afghanistan,AFG,Asia-Pacific,Economic Quality,Fiscal Sustainability,82,64,75,69,55,...,14.3,13.9,14.2,15.0,15.4,15.1,15.5,15.8,16.2,15.8
2,Afghanistan,AFG,Asia-Pacific,Economic Quality,Labour Force Engagement,130,128,127,122,117,...,8.2,8.3,8.6,8.8,8.9,8.9,9.1,9.2,9.3,9.3
3,Afghanistan,AFG,Asia-Pacific,Economic Quality,Macroeconomic Stability,131,65,13,3,7,...,5.8,6.5,7.3,7.0,6.9,5.8,5.3,5.4,5.0,4.9
4,Afghanistan,AFG,Asia-Pacific,Economic Quality,Productivity and Competitiveness,139,152,139,138,143,...,6.7,8.2,8.4,8.0,7.9,8.1,7.9,7.7,7.7,7.7


In [88]:
df2.columns

Index(['area_name', 'area_code', 'area_group', 'pillar_name', 'element_name',
       'rank_2009', 'rank_2010', 'rank_2011', 'rank_2012', 'rank_2013',
       'rank_2014', 'rank_2015', 'rank_2016', 'rank_2017', 'rank_2018',
       'rank_2019', 'score_2009', 'score_2010', 'score_2011', 'score_2012',
       'score_2013', 'score_2014', 'score_2015', 'score_2016', 'score_2017',
       'score_2018', 'score_2019'],
      dtype='object')

In [89]:
# keep only wanted columns
df2 = df2[['area_name','area_code','pillar_name','element_name','rank_2019','score_2019']]

In [90]:
df2.head()

,area_name,area_code,pillar_name,element_name,rank_2019,score_2019
0,Afghanistan,AFG,Economic Quality,Dynamism,146,2.5
1,Afghanistan,AFG,Economic Quality,Fiscal Sustainability,43,15.8
2,Afghanistan,AFG,Economic Quality,Labour Force Engagement,116,9.3
3,Afghanistan,AFG,Economic Quality,Macroeconomic Stability,119,4.9
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142,7.7


In [96]:
df2.rename(columns={'area_name':'country_name',
                    'rank_2019':'rank',       
                    'score_2019':'score'}, inplace=True)

In [97]:
df2.head()

,country_name,area_code,pillar_name,element_name,rank,score
0,Afghanistan,AFG,Economic Quality,Dynamism,146,2.5
1,Afghanistan,AFG,Economic Quality,Fiscal Sustainability,43,15.8
2,Afghanistan,AFG,Economic Quality,Labour Force Engagement,116,9.3
3,Afghanistan,AFG,Economic Quality,Macroeconomic Stability,119,4.9
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142,7.7


In [98]:
engine.table_names()

[]

In [99]:
df2.head()

,country_name,area_code,pillar_name,element_name,rank,score
0,Afghanistan,AFG,Economic Quality,Dynamism,146,2.5
1,Afghanistan,AFG,Economic Quality,Fiscal Sustainability,43,15.8
2,Afghanistan,AFG,Economic Quality,Labour Force Engagement,116,9.3
3,Afghanistan,AFG,Economic Quality,Macroeconomic Stability,119,4.9
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142,7.7


In [121]:
df3 = pd.read_csv('Resources/2lettercode.csv')

In [122]:
df3.head()


,English short name,French short name,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,Afghanistan (l'),AF,AFG,4
1,Albania,Albanie (l'),AL,ALB,8
2,Algeria,Algérie (l'),DZ,DZA,12
3,American Samoa,Samoa américaines (les),AS,ASM,16
4,Andorra,Andorre (l'),AD,AND,20


In [123]:
# keep only wanted columns
df3 = df3[['English short name','Alpha-3 code']]

In [124]:
df3.rename(columns={'English short name':'country_name',
                    'Alpha-3 code':'alpha_3_code',       
                    }, inplace=True)

In [125]:
df3.head()

,country_name,alpha_3_code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
